In [1]:
import seaborn as sns

In [2]:
df_train = pd.read_csv("../data/application_train.csv")
df_test = pd.read_csv("../data/application_test.csv")
df_y_train = df_train.loc[:, ['SK_ID_CURR', 'TARGET']]
df_y_test = df_test.loc[:, ['SK_ID_CURR', 'TARGET']]


/home/ray/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  after removing the cwd from sys.path.
/home/ray/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:1367: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [3]:
del df_train
del df_test

# Bureau和Bureau_bal处理

## 1. Bureau_bal 处理思路

1. 按照`SK_ID_BUREAU`分组计算数目，作为新变量`BUREAU_BAL_COUNT`
2. 对`STATUS`进行独热编码，发现大于0的情况非常稀疏，因此将其合并，得到`NOT_ZERO`
3. 对每个`SK_ID_BUREAU`求出closed情况的均值：` C_RATIO`
4. 合并`SK_ID_CURR`变量，并且按照`SK_ID_CURR`分组求所有变量的SUM和MEAN

## 2. BUREAU 处理思路

## Bureau_bal.csv

In [4]:
df_bureau = pd.read_csv('../data/bureau.csv')
df_bureau_bal = pd.read_csv('../data/bureau_balance.csv')

In [5]:
bureau_bal_count = df_bureau_bal.groupby('SK_ID_BUREAU')['SK_ID_BUREAU'].count()

In [6]:
df_bureau_bal = pd.get_dummies(df_bureau_bal)

In [7]:
df_bureau_bal = df_bureau_bal.groupby('SK_ID_BUREAU').sum()

In [8]:
df_bureau_bal['BUREAU_BAL_COUNT'] = df_bureau_bal.loc[:, ['STATUS_0', 'STATUS_1', 'STATUS_2', 
                                                          'STATUS_3', 'STATUS_4', 'STATUS_5', 'STATUS_C', 'STATUS_X']].sum(axis=1)

In [9]:
df_bureau_bal['NOT_ZERO'] = df_bureau_bal.loc[:, ['STATUS_1', 'STATUS_2', 'STATUS_3', 'STATUS_4', 'STATUS_5']].sum(axis=1)
df_bureau_bal['C_RATIO'] =df_bureau_bal['STATUS_C'] / df_bureau_bal['BUREAU_BAL_COUNT']

In [10]:
df_bureau_bal = df_bureau_bal.drop(columns=['STATUS_1', 'STATUS_2', 'STATUS_3', 'STATUS_4', 'STATUS_5'])

In [11]:
# df_bureau_bal = df_bureau_bal.drop(columns=['MONTHS_BALANCE'])

In [12]:
temp_dic = df_bureau.loc[:, ['SK_ID_CURR',	'SK_ID_BUREAU']]
temp_dic.index = temp_dic['SK_ID_BUREAU']
del temp_dic['SK_ID_BUREAU']
df_bureau_bal_res = pd.merge(df_bureau_bal, temp_dic, left_index=True, right_index=True,how='inner')

In [13]:
df_bureau_bal_res.head()

,MONTHS_BALANCE,STATUS_0,STATUS_C,STATUS_X,BUREAU_BAL_COUNT,NOT_ZERO,C_RATIO,SK_ID_CURR
SK_ID_BUREAU,,,,,,,,
5001710,-3403,5,48,30,83,0,0.578313,162368
5001711,-6,3,0,1,4,0,0.000000,162368
5001712,-171,10,9,0,19,0,0.473684,162368
5001713,-231,0,0,22,22,0,0.000000,150635
5001714,-105,0,0,15,15,0,0.000000,150635


In [14]:
temp1= df_bureau_bal_res.groupby('SK_ID_CURR').mean()
temp1.columns = temp1.columns.map(lambda x: x+'_MEAN')
temp2 = df_bureau_bal_res.groupby('SK_ID_CURR').sum()
temp2.columns = temp2.columns.map(lambda x: x+'_SUM')

## bureau.csv

In [15]:
df_bureau['CREDIT_ACTIVE'] = df_bureau['CREDIT_ACTIVE'].map(lambda x: 1 if x == 'Closed' else 0 if not pd.isnull(x) else x)

In [16]:
df_bureau['CREDIT_TYPE'] = df_bureau['CREDIT_TYPE'].map(lambda x: 'others' if x in ['Cash loan (non-earmarked)',
                                                                                   'Real estate loan', 'Loan for the purchase of equipment',
                                                                                   'Loan for purchase of shares (margin lending)','Unknown type of loan',
                                                                                   'Mobile operator loan', 'Interbank credit', 'Another type of loan', 
                                                                                   'Loan for working capital replenishment'] else x)

In [17]:
df_bureau = df_bureau.drop(columns = 'CREDIT_CURRENCY')

In [18]:
df_bureau = df_bureau.drop(columns = 'CREDIT_DAY_OVERDUE')

In [19]:
df_bureau = df_bureau.drop(columns = 'AMT_CREDIT_SUM_OVERDUE')

In [20]:
df_bureau = df_bureau.drop(columns = 'AMT_ANNUITY')

In [21]:
df_bureau = pd.get_dummies(df_bureau)

In [22]:
df_bureau = df_bureau.drop(columns = 'SK_ID_BUREAU')
df_bureau.head()

,SK_ID_CURR,CREDIT_ACTIVE,DAYS_CREDIT,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,DAYS_CREDIT_UPDATE,CREDIT_TYPE_Car loan,CREDIT_TYPE_Consumer credit,CREDIT_TYPE_Credit card,CREDIT_TYPE_Loan for business development,CREDIT_TYPE_Microloan,CREDIT_TYPE_Mortgage,CREDIT_TYPE_others
0,215354,1,-497,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,-131,0,1,0,0,0,0,0
1,215354,0,-208,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,-20,0,0,1,0,0,0,0
2,215354,0,-203,528.0,NaN,NaN,0,464323.5,NaN,NaN,-16,0,1,0,0,0,0,0
3,215354,0,-203,NaN,NaN,NaN,0,90000.0,NaN,NaN,-16,0,0,1,0,0,0,0
4,215354,0,-629,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,-21,0,1,0,0,0,0,0


In [23]:
temp3 = df_bureau.groupby('SK_ID_CURR').mean()
temp3.columns = temp3.columns.map(lambda x: x+'_MEAN')
temp4 = df_bureau.groupby('SK_ID_CURR').sum()
temp4.columns = temp4.columns.map(lambda x: x+'_SUM')


## Bureau 和 Bureau_bal汇总

In [24]:
df_train = pd.merge(left=df_y_train, right=temp1, how='left', left_on='SK_ID_CURR', right_index=True)
df_train = pd.merge(left=df_train, right=temp2, how='left', left_on='SK_ID_CURR', right_index=True)
df_train = pd.merge(left=df_train, right=temp3, how='left', left_on='SK_ID_CURR', right_index=True)
df_train = pd.merge(left=df_train, right=temp4, how='left', left_on='SK_ID_CURR', right_index=True)


In [25]:
df_test = pd.merge(left=df_y_test, right=temp1, how='left', left_on='SK_ID_CURR', right_index=True)
df_test = pd.merge(left=df_test, right=temp2, how='left', left_on='SK_ID_CURR', right_index=True)
df_test = pd.merge(left=df_test, right=temp3, how='left', left_on='SK_ID_CURR', right_index=True)
df_test = pd.merge(left=df_test, right=temp4, how='left', left_on='SK_ID_CURR', right_index=True)


In [26]:
print(df_train.shape)
print(df_test.shape)

(307511, 50)
(48744, 50)


In [27]:
TARGET = df_train['TARGET']

In [28]:
drop_num = []

In [29]:

def get_type(x):
    if isinstance(x[0], str):
        return 0
    if len( np.unique(x[:400].dropna())) < 15:
        return 0
    else:
        return 1
    
def plot(x):
    global TARGET
    _type = get_type(x)
    if _type == 0:
        discrete_plot(x, TARGET)
    else:
        continuous_plot(x, TARGET)
    
def discrete_plot(x, y):
    temp = pd.concat((x, y), axis=1)
    temp['value'] = 1
    temp = temp.dropna()
    temp.groupby(list(temp.columns)[:2]).sum().unstack().plot(kind='bar')
    print(temp.groupby([temp.columns[0]])['TARGET'].mean())
    print(temp.groupby([temp.columns[0]])['TARGET'].count())
    
def continuous_plot(x, y):
    x = x.dropna()
    sns.distplot(x.loc[y==0])
    sns.distplot(x.loc[y==1])

def mean_std(x):
    return (x - np.mean(x)) / np.std(x)

In [30]:
for col in df_train.columns[2:]:
    mean = np.mean(df_train[col])
    std = np.std(df_train[col])
    df_train[col] = (df_train[col] - mean) / std
    df_test[col] = (df_test[col] - mean) / std
    

In [31]:
df_train['MONTHS_BALANCE_MEAN_DUMMY'] = df_train['MONTHS_BALANCE_MEAN'].map(lambda x: 
                                                                                     1 if x > -0.6 else 0 if not pd.isnull(x) else x)
df_test['MONTHS_BALANCE_MEAN_DUMMY'] = df_test['MONTHS_BALANCE_MEAN'].map(lambda x: 
                                                                                     1 if x > -0.6 else 0 if not pd.isnull(x) else x)

In [32]:
df_train['STATUS_C_MEAN_DUMMY'] = df_train['STATUS_C_MEAN'].map(lambda x: 
                                                                                     1 if x > -0.3 else 0 if not pd.isnull(x) else x)
df_test['STATUS_C_MEAN_DUMMY'] = df_test['STATUS_C_MEAN'].map(lambda x: 
                                                                                     1 if x > -0.3 else 0 if not pd.isnull(x) else x)

In [33]:
df_train['BUREAU_BAL_COUNT_MEAN_DUMMY'] = df_train.iloc[:, 6].map(lambda x: 1  if x > -0.3 else 0 if not pd.isnull(x) else x)
df_train['BUREAU_BAL_COUNT_MEAN_DUMMY'] = df_test.iloc[:, 6].map(lambda x: 1 if x > -0.3 else 0 if not pd.isnull(x) else x)


In [34]:
df_train.iloc[:, 9] = df_train.iloc[:, 9].map(lambda x: -3 if x <= -3 else x)
df_test.iloc[:, 9] = df_test.iloc[:, 9].map(lambda x: -3 if x <= -3 else x)


In [35]:
df_train.iloc[:, 10] = df_train.iloc[:, 10].map(lambda x: 3 if x >=3 else x)
df_test.iloc[:, 10] = df_test.iloc[:, 10].map(lambda x: 3 if x >= 3 else x)


In [36]:
df_train.iloc[:, 11] = df_train.iloc[:, 11].map(lambda x: 2.5 if x >=2.5 else x)
df_test.iloc[:, 11] = df_test.iloc[:, 11].map(lambda x: 2.5 if x >= 2.5 else x)

In [37]:
df_train.iloc[:, 12] = df_train.iloc[:, 12].map(lambda x: 2 if x >=2 else x)
df_test.iloc[:, 12] = df_test.iloc[:, 12].map(lambda x: 2 if x >= 2 else x)

In [38]:
df_train.iloc[:, 13] = df_train.iloc[:, 13].map(lambda x: 2.5 if x >=2.5 else x)
df_test.iloc[:, 13] = df_test.iloc[:, 13].map(lambda x: 2.5 if x >= 2.5 else x)

In [39]:
df_train.iloc[:, 11] = df_train.iloc[:, 11].map(lambda x: 2.5 if x >=2.5 else x)
df_test.iloc[:, 11] = df_test.iloc[:, 11].map(lambda x: 2.5 if x >= 2.5 else x)

In [40]:
df_train['NOT_ZERO_SUM_DUMMY'] = df_train['NOT_ZERO_SUM'].map(lambda x: '2' if x > 0.50 
                                                              else '1' if x > -0.2 else '0' if not pd.isnull(x) else x)
df_test['NOT_ZERO_SUM_DUMMY'] = df_test['NOT_ZERO_SUM'].map(lambda x: '2' if x > 0.50 
                                                              else '1' if x > -0.2 else '0' if not pd.isnull(x) else x)

In [41]:
df_train['CREDIT_ACTIVE_MEAN_DUMMY'] = df_train['CREDIT_ACTIVE_MEAN'].map(lambda x: 
                                                                          1 if x > -0.5 else 0 if not pd.isnull(x) else x)
df_test['CREDIT_ACTIVE_MEAN_DUMMY'] = df_test['CREDIT_ACTIVE_MEAN'].map(lambda x: 
                                                                          1 if x > -0.5 else 0 if not pd.isnull(x) else x)

In [42]:
df_train['DAYS_CREDIT_MEAN_DUMMY'] = df_train['DAYS_CREDIT_MEAN'].map(lambda x: 
                                                                         1 if x >= 0.4 else 0 if not pd.isnull(x) else x)
df_test['DAYS_CREDIT_MEAN_DUMMY'] = df_test['DAYS_CREDIT_MEAN'].map(lambda x: 
                                                                          1 if x >= 0.4 else 0 if not pd.isnull(x) else x)

In [43]:
df_train.iloc[:, 18] = df_train.iloc[:, 18].map(lambda x: 1 if x >=1 else 0 if not pd.isnull(x) else x)
df_test.iloc[:, 18] = df_test.iloc[:, 18].map(lambda x: 1 if x >= 1 else 0 if not pd.isnull(x) else x)

In [44]:
df_train.iloc[:, 19] = df_train.iloc[:, 19].map(lambda x: -3 if x <= -3 else x)
df_test.iloc[:, 19] = df_test.iloc[:, 19].map(lambda x: -3 if x <= -3 else x)

df_train['DAYS_ENDDATE_FACT_MEAN_DUMMY'] = df_train.iloc[:, 19].map(lambda x: 1 if x >0.4 else 0 if not pd.isnull(x) else x)
df_test['DAYS_ENDDATE_FACT_MEAN_DUMMY'] = df_test.iloc[:, 19].map(lambda x: 1 if x >0.4 else 0 if not pd.isnull(x) else x)

In [45]:
drop_num += [20, 21, 22, 23, 24, 26, 27, 28, 29, 30, 31, 32]

In [46]:
df_train.iloc[:, 22] = df_train.iloc[:, 22].map(lambda x: -3 if x <= -3 else x)
df_test.iloc[:, 22] = df_test.iloc[:, 22].map(lambda x: -3 if x <= -3 else x)

df_train['DAYS_CREDIT_UPDATE_MEAN_DUMMY'] = df_train.iloc[:, 22].map(lambda x: 1 if x >0.4 else 0 if not pd.isnull(x) else x)
df_test['DAYS_CREDIT_UPDATE_MEAN_DUMMY'] = df_test.iloc[:, 22].map(lambda x: 1 if x >0.4 else 0 if not pd.isnull(x) else x)

In [47]:
df_train.iloc[:, 34] = df_train.iloc[:, 34].map(lambda x: -3 if x <= -3 else x)
df_test.iloc[:, 34] = df_test.iloc[:, 34].map(lambda x: -3 if x <= -3 else x)

df_train['DAYS_CREDIT_UPDATE_MEAN_DUMMY'] = df_train.iloc[:, 34].map(lambda x: 1 if x >0.4 else 0 if not pd.isnull(x) else x)
df_test['DAYS_CREDIT_UPDATE_MEAN_DUMMY'] = df_test.iloc[:, 34].map(lambda x: 1 if x >0.4 else 0 if not pd.isnull(x) else x)

In [48]:
df_train.iloc[:, 35] = df_train.iloc[:, 35].map(lambda x: -1 if x <= -1 else 3 if x >= 3 else x)
df_test.iloc[:, 35] = df_test.iloc[:, 35].map(lambda x: -1 if x <= -1 else 3 if x >= 3 else x)


In [49]:
df_train.iloc[:, 36] = df_train.iloc[:, 36].map(lambda x: -1.5 if x <= -1.5 else x)
df_test.iloc[:, 36] = df_test.iloc[:, 36].map(lambda x: -1.5 if x <= -1.5 else x)


In [50]:
drop_num.append(37)

In [51]:
df_train.iloc[:, 38] = df_train.iloc[:, 38].map(lambda x: 1 if x < 0 else 0 if not pd.isnull(x) else x)
df_test.iloc[:, 38] = df_test.iloc[:, 38].map(lambda x: 1 if x < 0 else 0 if not pd.isnull(x) else x)


In [52]:
drop_num+=[39, 40, 41, 42, 46, 49]

In [53]:
df_train.iloc[:, 43] = df_train.iloc[:, 43].map(lambda x: 1 if x < 0 else 0 if not pd.isnull(x) else x)
df_test.iloc[:, 43] = df_test.iloc[:, 43].map(lambda x: 1 if x < 0 else 0 if not pd.isnull(x) else x)

In [54]:
df_train['CREDIT_TYPE_Credit card_SUM_DUMMY1'] = df_train.iloc[:, 45].map(lambda x: 1 if x < -0.5 else 0  if not pd.isnull(x) else x)
df_test['CREDIT_TYPE_Credit card_SUM_DUMMY1'] = df_test.iloc[:, 45].map(lambda x: 1 if x < -0.5 else 0 if not pd.isnull(x) else x)

df_train['CREDIT_TYPE_Credit card_SUM_DUMMY2'] = df_train.iloc[:, 45].map(lambda x: 1 if -0.5 < x < 0 else 0  if not pd.isnull(x) else x)
df_test['CREDIT_TYPE_Credit card_SUM_DUMMY2'] = df_test.iloc[:, 45].map(lambda x: 1 if -0.5 < x < 0 else 0 if not pd.isnull(x) else x)

df_train['CREDIT_TYPE_Credit card_SUM_DUMMY3'] = df_train.iloc[:, 45].map(lambda x: 1 if x >= 0 else 0  if not pd.isnull(x) else x)
df_test['CREDIT_TYPE_Credit card_SUM_DUMMY3'] = df_test.iloc[:, 45].map(lambda x: 1 if x >=0 else 0 if not pd.isnull(x) else x)

In [55]:
df_train.iloc[:, 47] = df_train.iloc[:, 47].map(lambda x: 1 if x < 0 else 0 if not pd.isnull(x) else x)
df_test.iloc[:, 47] = df_test.iloc[:, 47].map(lambda x: 1 if x < 0 else 0 if not pd.isnull(x) else x)


In [56]:
df_train.iloc[:, 48] = df_train.iloc[:, 48].map(lambda x: 1 if x < 0 else 0 if not pd.isnull(x) else x)
df_test.iloc[:, 48] = df_test.iloc[:, 48].map(lambda x: 1 if x < 0 else 0 if not pd.isnull(x) else x)


In [57]:
temp_value = df_train.iloc[:, 45].value_counts().keys()
dic = dict(list(zip(list(temp_value), list(range(len(temp_value))))))
df_train.iloc[:, 45] = df_train.iloc[:, 45].map(dic)

temp_value = df_test.iloc[:, 45].value_counts().keys()
dic = dict(list(zip(list(temp_value), list(range(len(temp_value))))))
df_test.iloc[:, 45] = df_test.iloc[:, 45].map(dic)

In [58]:
df_train.iloc[:, 45] = df_train.iloc[:, 45].map(lambda x: str(int(x)) if x < 5 else '5' if not pd.isnull(x) else x)
df_test.iloc[:, 45] = df_test.iloc[:, 45].map(lambda x: str(int(x)) if x < 5 else '5' if not pd.isnull(x) else x)

In [59]:
df_train = df_train.drop(columns=df_train.columns[drop_num])
df_test = df_test.drop(columns=df_test.columns[drop_num])

In [60]:
df_train.to_csv('../data/handled/bureau_train.csv')

In [61]:
df_test.to_csv('../data/handled/bureau_test.csv')